In [4]:
from bs4 import BeautifulSoup
from urllib.error import HTTPError
import urllib.request
import requests
import pandas as pd
import re
import csv

def crawl_page(link=''):
    # req = urllib.request.Request(
    #         link, 
    #         data=None, 
    #         headers={
    #             'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
    #             'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.79 Safari/537.36 Edge/14.14393'
    #         }
    # )
    # try:
    #     page= urllib.request.urlopen(req)
    #     return_code= page.getcode()
    #     soup_page= BeautifulSoup(page,'html.parser')
    #     return {'page':soup_page, 'status': return_code}
    # except HTTPError as err:
    #     if err.code == 404:
    #         return_code=404
    #         return {'page':'no page', 'status': return_code}
    #     else:
    #         raise
    try:
        response = requests.get(
            url='http://api.scraperapi.com', #scrapeapi
            params={
                'api_key': 'e3458ff579ef5c1369f185af0ceaecfb',
                'url': link,  
            },
            
        )
        return_code= 200
        soup_page= BeautifulSoup(response.content,'html.parser')
        return {'page':soup_page, 'status': return_code}
    except requests.exceptions.HTTPError as err:
        if err.response.status_code == 404:
            return_code=404
            return {'page':'no page', 'status': return_code}
    

def write_csv(line=[], file=''):
    with open(file, 'a', newline='') as csvfile:
        spamwriter = csv.writer(csvfile, delimiter=' ',
                                quotechar='|', quoting=csv.QUOTE_MINIMAL)
        spamwriter.writerow(line)

def remove_row_csv():
    with open('temp_27.csv') as inp:
        for link in csv.reader(inp):
            count=0
            temp=link
            with open('used_estate.txt') as out:
                for estate in csv.reader(out):
                    if re.match("(.*)/"+estate[0],temp[1]) is not None:
                        count+=1
            if count==0:
                new_row= [link[0]+','+link[1]]
                write_csv(new_row,"temp_28.csv")

def create_urls(file=''):
    with open(file) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 1
        for row in csv_reader:
            new_row= [str(line_count)+','+row[0]]
            write_csv(new_row,'property_url.csv')
            line_count+=1

    

In [13]:
class DataCollect():
    def __init__(self):
        self.__url_to_open='https://www.property24.com'
        self.__soup=crawl_page(self.__url_to_open)['page']
        self.__area_url=[]
        self.__list_url=[]
        self.__urls=pd.read_csv("temp_43_v1.csv",usecols= ['id','url']) 
    
    def collect_all_first_page_url(self):
        other_links= self.__soup.find('div',class_='p24_popular').find(class_='col-8').find_all('a',class_='p24_bold')
        for link in other_links:
            self.__area_url.append(self.__url_to_open+link.get('href'))
        first_element_tag= self.__soup.find('div',class_='p24_popular').find(class_='col-8').find_all(class_='row')[1]
        first_link=first_element_tag.find('a').get('href')
        temp_soup= crawl_page(self.__url_to_open+first_link)['page']
        parent_link= temp_soup.find(id='breadCrumbContainer').find_all('a')[1]
        self.__area_url.append(self.__url_to_open+parent_link.get('href'))
    
    def collect_all_url(self):
        self.collect_all_first_page_url()
        for link in self.__area_url:
            soup= crawl_page(link)['page']
            link_house_tag= soup.find('ul',class_='p24_relatedSales').find(href=re.compile("houses-for-sale")).get('href')
            link_apart_tag= soup.find('ul',class_='p24_relatedSales').find(href=re.compile("apartments-for-sale")).get('href')
            link_townhouse_tag= soup.find('ul',class_='p24_relatedSales').find(href=re.compile("townhouses-for-sale")).get('href')
            self.__list_url.append(link_house_tag)
            self.__list_url.append(link_apart_tag)
            self.__list_url.append(link_townhouse_tag)

    def get_property_data(self, link=''):
        data={}

        estate_id= link.split('/')[-1].split('?')[0]

        prop_soup_element= crawl_page(self.__url_to_open+link)

        
        if prop_soup_element['status'] !=200:
            print('heres')
            return False
        
        prop_soup= prop_soup_element['page']

        if prop_soup is None:
            print('here')
            return False

        #check if page not found
        check_page_not_found_msg= prop_soup.select_one('.p24_not_found')
        if check_page_not_found_msg is not None:
            print("====> Page not found !!")
            return False
        
        #check page expired
        check_page_expired = prop_soup.select_one('.p24_results.p24_expired')
        if check_page_expired is not None:
            print("====> Page expired !!")
            return False
        

        name= prop_soup.find('div',class_='p24_listingFeaturesWrapper').find('h1').get_text()
        data['name']=name

        price= prop_soup.find('div',class_='p24_listingFeaturesWrapper').find('div',class_="p24_price").get_text()
        data['price']=price

        province= link.split('/')[4]
        data['province']=province

        city= link.split('/')[3]
        data['city']=city


        is_adress= len(prop_soup.find('div',class_='p24_listingFeaturesWrapper').find_all('div',class_="p24_mBM")) >= 3
        if is_adress:
            if len(prop_soup.find('div',class_='p24_listingFeaturesWrapper').find_all('div',class_="p24_mBM")) == 3:
                index=-1
            else:
                index=3
            address= prop_soup.find('div',class_='p24_listingFeaturesWrapper').find_all('div',class_="p24_mBM")[index].get_text()
            data['address']=address
        else:
            address=  prop_soup.select_one('.p24_addressPropOverview').get_text()
            data['address']=address

        #property overview
        if type(prop_soup.select_one('.p24_propertyOverview')) is not None:
            prop_overview_panel= prop_soup.find('div',class_='p24_propertyOverview').find('div',class_='panel-body')
            prop_overview_row= prop_overview_panel.find_all('div',class_='p24_propertyOverviewRow')
            for prop in prop_overview_row:
                key= prop.find('div',class_='p24_propertyOverviewKey').get_text()
                # value= prop.select_one('.noPadding > .p24_info').get_text()
                value= prop.find('div',class_='noPadding').contents[1].get_text()
                data[key]=value

        #rooms
        rooms_panel=prop_soup.select_one('#js_accordion_rooms')
        if rooms_panel is not None:
            prop_overview_row= rooms_panel.find_all('div',class_='p24_propertyOverviewRow')
            for prop in prop_overview_row:
                key= prop.find('div',class_='p24_propertyOverviewKey').get_text()
                # value= prop.select_one('.noPadding > .p24_info').get_text()
                value= prop.find('div',class_='noPadding').contents[1].get_text()
                data[key]=value

        #external_features
        external_features_panel=prop_soup.select_one('#js_accordion_externalfeatures')
        if external_features_panel is not None:
            prop_overview_row= external_features_panel.find_all('div',class_='p24_propertyOverviewRow')
            for prop in prop_overview_row:
                key= prop.find('div',class_='p24_propertyOverviewKey').get_text()
                # value= prop.select_one('.noPadding > .p24_info').get_text()
                value= prop.find('div',class_='noPadding').contents[1].get_text()
                data[key]=value
        
        #other_features
        other_features_panel=prop_soup.select_one('#js_accordion_otherfeatures')
        if other_features_panel is not None:
            prop_overview_row= other_features_panel.find_all('div',class_='p24_propertyOverviewRow')
            for prop in prop_overview_row:
                key= prop.find('div',class_='p24_propertyOverviewKey').get_text()
                # value= prop.select_one('.noPadding > .p24_info').get_text()
                value= prop.find('div',class_='noPadding').contents[1].get_text()
                data[key]=value

        #building_features
        building_features_panel=prop_soup.select_one('#js_accordion_building')
        if building_features_panel is not None:
            prop_overview_row= building_features_panel.find_all('div',class_='p24_propertyOverviewRow')
            for prop in prop_overview_row:
                key= prop.find('div',class_='p24_propertyOverviewKey').get_text()
                # value= prop.select_one('.noPadding > .p24_info').get_text()
                value= prop.find('div',class_='noPadding').contents[1].get_text()
                data[key]=value

        #points_interest
        interest_soup= crawl_page(self.__url_to_open+'/ListingReadOnly/PointsOfInterestForListing?ListingNumber=P24-'+estate_id)
        # points_interest_panel=prop_soup.select_one('#accordian-points-of-interest')
        # print(points_interest_panel)
        if interest_soup['status'] == 200:
            categories= interest_soup['page'].select('.js_P24_POICategory')
            for cat in categories:
                interest= cat.select_one('.row > .col-6 > .p24_semibold')
                distances= cat.select('.row > .col-6.noPadding.p24_semibold')
                dist_tab=[]
                for dist in distances:
                    dist_tab.append(dist.get_text())
                data[interest.get_text()]=dist_tab

        print(str(data))
        write_csv([str(data)],'data_property.csv')

        
        return True

    def crawl_property_per_page(self,city,page,number_properties):
        link=self.__url_to_open+city+'/p'+str(page)
        print('==>Page link: ',link)
        cur_soup= crawl_page(link)['page']
        properties= cur_soup.find_all('div',class_='js_resultTile')
        for prop_index in range(len(properties)-1):
            prop_link= properties[prop_index].find('a').get('href')
            write_csv([prop_link],'data_property_url_v1.csv')

        number_properties+= len(properties)
        return number_properties

    def crawl_property_per_city(self,city,cur_page):
        soup= crawl_page(self.__url_to_open+city)['page']
        pagination= soup.find('div',class_='p24_pager').find_all('li')[-1].find('a').get('data-pagenumber')
        page_number= int(pagination)
        number_properties=0
        print("City url: ",city)
        for page in range(cur_page, page_number+1):
            print("=>Page number: ",page)
            number_properties = self.crawl_property_per_page(city,page,number_properties)
        print(number_properties)
    
    def crawl_properties(self):
        #shuffle
        self.__urls = self.__urls.sample(frac = 1).reset_index(drop=True)
        for i in range(self.__urls.size):
            prop_link= self.__urls.iloc[i]['url']
            
            print('==>Property link: ', prop_link)
            check_property_crawl = self.get_property_data(prop_link)
            if check_property_crawl:
                self.__urls= self.__urls.drop(self.__urls.index[i])
                self.__urls.to_csv('temp_44_v1.csv',index=False)

    def crawl_property_url(self):
        self.__list_url=['/townhouses-for-sale/gauteng/1']
        for link in self.__list_url:
            self.crawl_property_per_city(link,1)

In [14]:
dc= DataCollect()
try:
    dc.crawl_properties()
except:
    dc.crawl_properties()

==>Property link:  /for-sale/chanteclair/hermanus/western-cape/7855/109942678
{'name': '5 Bedroom House for Sale in Chanteclair', 'price': 'R 7 800 000', 'province': 'western-cape', 'city': 'hermanus', 'address': '\n5 Bedroom House for Sale in Chanteclair\n', 'Listing Number': '109942678', 'Type of Property': 'House', 'Street Address': '32 Black Eagle Road, Chanteclair', 'Lifestyle': 'Suburban', 'Listing Date': '11 May 2021', 'Erf Size': '1 293 m²', 'Floor Size': '708 m²', 'Age': '1', 'Pets Allowed': 'Yes', 'Bedrooms': '5', 'Bathrooms': '5', 'Kitchen': 'Breakfast Nook, Open Plan, Scullery, Laundry, Stove, Extractor Fan, Dishwasher Connection, Granite Tops', 'Office': '1', 'Family/TV Room': 'Family TV Room', 'Rooms': 'Patio, Blinds, TV Port, Veranda', 'Garage': '3', 'Parking': '5', 'Garden': 'Yes', 'Pool': 'Yes', 'Security': 'Electric Garage, Electric Gate, Intercom, Perimeter Wall', 'Temperature Control': 'Temperature control features', 'Special Feature': 'TV Antenna, Water Tanks', 'St

==>Property link:  /for-sale/hatfield/pretoria/gauteng/304/110251669
{'name': '6 Bedroom Townhouse for Sale in Hatfield', 'price': 'R 2 250 000', 'province': 'gauteng', 'city': 'pretoria', 'address': '\n6 Bedroom Townhouse for Sale in Hatfield\n', 'Listing Number': '110251669', 'Type of Property': 'Townhouse', 'Street Address': ' 3 SS DEL ESTE, 1090 Francis Baard Street, Hatfield', 'Description': 'Duplex', 'Lifestyle': 'Complex', 'Listing Date': '30 July 2021', 'Floor Size': '251 m²', 'Levies': 'R 800', 'Rates and Taxes': 'R 3 100', 'Pets Allowed': 'No', 'Bedrooms': '6', 'Bedroom 1': ' ', 'Bedroom 2': ' ', 'Bedroom 3': ' ', 'Bedroom 4': ' ', 'Bedroom 5': ' ', 'Bedroom 6': ' ', 'Bathrooms': '3', 'Kitchen': ' ', 'Lounge': ' ', 'Dining Room': ' ', 'Garage': '2', 'Garage 1': ' ', 'Garage 2': ' ', 'Garden': 'Yes', 'Pool': 'Yes', 'Security': 'Electric fencing', 'Transport and Public Services': ['0.11km', '0.19km', '0.22km', '0.39km'], 'Food and Entertainment': ['0.23km', '0.23km', '0.23km', 

==>Property link:  /for-sale/annlin-wes/pretoria/gauteng/32916/109595192
{'name': '2 Bedroom Apartment / Flat for Sale in Annlin-wes', 'price': 'R 840 000', 'province': 'gauteng', 'city': 'pretoria', 'address': 'Annlin-Wes, Pretoria', 'Listing Number': '109595192', 'Type of Property': 'Apartment / Flat', 'Listing Date': '04 February 2021', 'Floor Size': '87 m²', 'Price per m²': 'R 9 655', 'Levies': 'R 800', 'Rates and Taxes': 'R 650', 'Age': '6', 'Pets Allowed': 'No', 'Bedrooms': '2', 'Bathrooms': '1', 'Kitchen': 'Open Plan, Laundry, Under Counter Oven', 'Parking': '2', 'Garden': 'Yes', 'Security': 'Perimeter Wall', 'Style': 'Contemporary', 'Facing': 'North', 'Roof': 'Insulation', 'Wall': 'Face-brick', 'Floor': 'Tiled Floors', 'Education': ['0.64km', '0.91km', '1.04km', '1.28km', '1.72km', '1.72km', '2.08km', '2.19km', '2.27km', '2.39km', '2.48km', '2.71km', '3.22km'], 'Transport and Public Services': ['0.73km', '0.85km', '1.74km', '1.80km', '2.61km', '2.75km', '2.88km'], 'Sports and L

==>Property link:  /for-sale/jeffreys-bay-central/jeffreys-bay/eastern-cape/10503/109542303
{'name': '3 Bedroom Apartment / Flat for Sale in Jeffreys Bay Central', 'price': 'R 1 780 000', 'province': 'eastern-cape', 'city': 'jeffreys-bay', 'address': 'Jeffreys Bay Central, Jeffreys Bay', 'Listing Number': '109542303', 'Type of Property': 'Apartment / Flat', 'Listing Date': '22 January 2021', 'Floor Size': '168 m²', 'Price per m²': 'R 10 595', 'Pets Allowed': 'No', 'Furnished': 'Yes', 'Bedrooms': '3', 'Bathrooms': '3', 'Kitchens': '1', 'Lounges': '1', 'Dining Rooms': '1', 'Garage': '1', 'Education': ['0.67km', '1.56km', '1.63km', '1.97km'], 'Transport and Public Services': ['3.11km', '5.64km', '7.42km', '8.07km'], 'Food and Entertainment': ['3.45km', '3.57km']}
==>Property link:  /for-sale/noordheuwel/krugersdorp/gauteng/940/110166408
==>Property link:  /for-sale/klerksdorp-central/klerksdorp/north-west/25377/109840526
{'name': '1 Bedroom Apartment / Flat for Sale in Klerksdorp Central'

==>Property link:  /for-sale/komdraai/witbank/mpumalanga/28035/110091486
{'name': '3 Bedroom Townhouse for Sale in Komdraai', 'price': 'R 1 150 000', 'province': 'mpumalanga', 'city': 'witbank', 'address': 'Komdraai, Witbank', 'Listing Number': '110091486', 'Type of Property': 'Townhouse', 'Listing Date': '14 June 2021', 'Levies': 'R 1 249', 'Bedrooms': '3', 'Bedroom 1': 'Tiled Floors, Curtain Rails, Built-in Cupboards, King Bed', 'Bedroom 2': 'Tiled Floors, Curtain Rails, Built-in Cupboards, Queen Bed', 'Bedroom 3': 'Tiled Floors, Curtain Rails, Built-in Cupboards, Double Bed', 'Bathrooms': '3', 'Bathroom 1': 'Tiled Floors, Curtain Rails, Shower, Toilet and Basin', 'Bathroom 2': 'Tiled Floors, Curtain Rails, Bath, Toilet and Basin', 'Bathroom 3': 'Tiled Floors, Main en Suite, Curtain Rails, Shower, Toilet and Basin', 'Kitchen': 'Breakfast Nook, Open Plan, Extractor Fan, Washing Machine Connection, Tiled Floors, Built-in Cupboards, Under Counter Oven', 'Lounge': 'Patio, Tiled Floors, C

==>Property link:  /for-sale/savanna-city/walkerville/gauteng/16142/109684838
{'name': '2 Bedroom House for Sale in Savanna City', 'price': 'R 545 703', 'province': 'gauteng', 'city': 'walkerville', 'address': '\n2 Bedroom House for Sale in Savanna City\n', 'Listing Number': '109684838', 'Type of Property': 'House', 'Street Address': ' 316 Town Square Boulevard, Savanna City', 'Description': 'New Development, Freestanding', 'Lifestyle': 'Security Complex, Suburban', 'Listing Date': '25 February 2021', 'Erf Size': '250 m²', 'Floor Size': '40 m²', 'No Transfer Duty': 'Yes', 'Rates and Taxes': 'R 200', 'Pets Allowed': 'Yes', 'Coverage': '16 %', 'Bedrooms': '2', 'Bedroom 1': 'Curtain Rails, Double Bed', 'Bedroom 2': 'Curtain Rails, Single Bed', 'Bathroom': 'Full, Shower, Curtain Rails, Bath, Toilet and Basin', 'Kitchen': 'Open Plan, Curtain Rails', 'Parking': '2', 'Parking 1': 'Secure Parking, Triple', 'Parking 2': 'Visitors Parking', 'Security': 'Security Gate, Perimeter Wall, Boomed Area

==>Property link:  /for-sale/montana/pretoria/gauteng/3984/109875832
{'name': '3 Bedroom Apartment / Flat for Sale in Montana', 'price': 'R 995 000', 'province': 'gauteng', 'city': 'pretoria', 'address': 'Montana, Pretoria', 'Listing Number': '109875832', 'Type of Property': 'Apartment / Flat', 'Listing Date': '18 April 2021', 'Floor Size': '130 m²', 'Price per m²': 'R 7 654', 'Levies': 'R 2 050', 'Rates and Taxes': 'R 810', 'Pets Allowed': 'Yes', 'Bedrooms': '3', 'Bedroom 1': 'Carpets', 'Bedroom 2': 'Carpets', 'Bedroom 3': 'Carpets', 'Bathrooms': '2', 'Bathroom 1': 'Tiled Floors', 'Bathroom 2': 'Tiled Floors', 'Kitchen': 'Open Plan', 'Office/study': 'Open Plan', 'Garage': '2', 'Garage 1': 'Single, Covered', 'Garage 2': 'Single, Covered', 'Garden': 'Garden Terrace', 'Security': 'Electric Gate, Security Gate, Burglar Bars', 'Nearby Public Transport': 'Bus, Minibus Taxi', 'Internet Access': 'Fibre', 'Wheelchair Accessible': 'No', 'Generator': 'No', 'Backup Water': 'No', 'Standalone Build

==>Property link:  /for-sale/umbilo/durban/kwazulu-natal/6029/109943813
{'name': '1 Bedroom Apartment / Flat for Sale in Umbilo', 'price': 'R 495 000', 'province': 'kwazulu-natal', 'city': 'durban', 'address': '\n1 Bedroom Apartment / Flat for Sale in Umbilo\n', 'Listing Number': '109943813', 'Type of Property': 'Apartment / Flat', 'Street Address': '4 Hurley Road, Umbilo', 'Lifestyle': 'Security Complex', 'Listing Date': '06 May 2021', 'Erf Size': '1 132 m²', 'Floor Size': '55 m²', 'Price per m²': 'R 9 000', 'Levies': 'R 1 100', 'Rates and Taxes': 'R 293', 'Bedroom': '1', 'Bathrooms': '1', 'Kitchens': '1', 'Garage': '1', 'Education': ['0.22km', '0.23km', '0.32km', '0.37km', '0.62km', '0.63km', '0.86km', '0.87km', '0.94km', '1.18km', '1.50km', '1.71km', '2.00km', '2.13km', '2.17km', '2.57km', '2.58km', '2.59km', '2.71km', '2.83km', '2.83km', '2.97km'], 'Health': ['0.70km', '0.77km', '2.97km', '2.97km'], 'Transport and Public Services': ['1.04km', '1.32km', '1.95km', '2.63km']}
==>Prope

==>Property link:  /for-sale/equestria/pretoria/gauteng/266/108974988
{'name': '3 Bedroom Townhouse for Sale in Equestria', 'price': 'R 2 195 000', 'province': 'gauteng', 'city': 'pretoria', 'address': 'Equestria, Pretoria', 'Listing Number': '108974988', 'Type of Property': 'Townhouse', 'Lifestyle': 'Retirement Living', 'Listing Date': '11 August 2020', 'Erf Size': '151 m²', 'Floor Size': '151 m²', 'Levies': 'R 2 337', 'No Transfer Duty': 'Yes', 'Age': '3', 'Bedrooms': '3', 'Bedroom 1': 'Bedroom 3 with en-suite bathroom, built-in cupboards and laminate wood floors', 'Bedroom 2': 'Bedroom 2 with built-in cupboards and laminate wood floors', 'Bedroom 3': 'Main bedroom with en-suite bathroom, built-in cupboards and laminate wood floors', 'Bathrooms': '2', 'Bathroom 1': 'Bathroom 1 with basin, bath and toilet', 'Bathroom 2': 'Bathroom 2 with basin, shower and toilet', 'Kitchen': 'Kitchen with tiled floors, caesar stone finishes and dish-wash machine connection', 'Reception Rooms': '2', 'O

KeyboardInterrupt: 